Задача 1. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId


In [1]:
import pandas as pd

In [2]:
dfhw8_1 = pd.read_csv('C:\\Users\\yusuper\\Documents\\GitHub\\pyda5_netology\\ratings.csv')
dfhw8_1.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [10]:
ratings_grouped = dfhw8_1.groupby('userId').count().reset_index()
ratings_filtered = ratings_grouped[ratings_grouped['rating']>=100].head()
ratings_filtered.head()
film_fans_user_ids = ratings_filtered['userId'].tolist()
df8_1 = dfhw8_1[ dfhw8_1['userId'].isin(film_fans_user_ids) ] # отсортированный

In [35]:
a = df8_1[['userId','timestamp']].groupby('userId').max() # - переменая максимума  
a = a.rename(columns = {'timestamp':'timestamp_max'})
a

,timestamp_max
userId,
1,965719662
4,1007574542
6,845556915
7,1176181731
10,1455619275


In [36]:
b = df8_1[['userId','timestamp']].groupby('userId').min() # - переменая минимума
b = b.rename(columns = {'timestamp':'timestamp_min'})
b

,timestamp_min
userId,
1,964980499
4,945078428
6,845553109
7,1106635416
10,1455301553


In [46]:
c = a.join(b, how='left')
c = c[['timestamp_max']].sub(c['timestamp_min'], axis=0)
c = c.rename(columns = {'timestamp_max':'life_time'})
c

,life_time
userId,
1,739163
4,62496114
6,3806
7,69546315
10,317722


2. Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.
К домашнему заданию, задача 2¶
Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов


In [1]:
import pandas as pd

In [2]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [3]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [4]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [5]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [6]:
all_transport = rzd.merge(auto, how = 'outer', on = 'client_id' ).merge(air, how = 'outer', on = 'client_id' )
all_transport

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [7]:
all_transport_address = all_transport.merge(client_base, how = 'outer', on = 'client_id' )
all_transport_address

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


3. В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок?

Анализировал бы какие товары и с какой частотой покупает конкретный юзер в течение дня и в разные дни недели. Очевидно, что вечером он покупает продукты домой, на выходных продукты на неделю. Днем в будни скорее всего или ест бизнес ланч или покупает салатики на обед. Естесственно средний чек разный. Можно подсказывать какие продукты купить, какие докупить и тд. Надеюсь я правильно понял задание. 